In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sample_dh.csv')

In [3]:
df

Date  Vertical                                  Author  \
0    JAN 01 2017, 08:25 IST  Business              Mumbai, Dec 31, 2017, PTI,   
1    JAN 01 2017, 10:23 IST  Business            New Delhi, Jan 1, 2016, PTI,   
2    JAN 01 2017, 14:54 IST  Business           New Delhi, Jan 1, 2017 (PTI),   
3    JAN 01 2017, 15:36 IST  Business           New Delhi, Jan 1, 2017 (PTI),   
4    JAN 01 2017, 16:06 IST  Business           New Delhi, Jan 1, 2017 (PTI),   
..                      ...       ...                                     ...   
195  JAN 02 2017, 17:47 IST  National            New Delhi, Jan 2, 2016, PTI,   
196  JAN 02 2017, 18:38 IST  National          New Delhi, Jan 2, 2016, (PTI),   
197  JAN 02 2017, 18:50 IST  National              Chennai, Jan 2, 2017, PTI,   
198  JAN 02 2017, 19:37 IST  National            New Delhi, Jan 2, 2017, PTI,   
199  JAN 02 2017, 21:02 IST  National  New Delhi/Lucknow, Jan 2, 2016, (PTI),   

                                              Headline  \
0    NRIs can exchange defunct notes till June 30, ...   
1    Govt mulls 10 paise cess to provide social saf...   
2    New Year bonanza: Jobs mkt eyes 8.75 L hiring ...   
3    Can't give info if FM, CEA consulted before de...   
4    Jan Dhan A/c deposits double to Rs 87K cr, I-T...   
..                                                 ...   
195  Service charge by hotels/restaurants not manda...   
196  NRIs, Indians returning home to show old notes...   
197       Not in favour of exhuming Jaya's body: Naidu   
198  Prepared for a two-front war but want peace: A...   
199  SP's war moves to Delhi; Mulayam approaches EC...   

                                           Description  
0    RBI tonight came out with conditions for excha...  
1    Railway coolies -- roughly 20,000 -- may soon ...  
2    The New Year is expected to bring good news fo...  
3    Whether views of the Chief Economic Advisor an...  
4    Deposits in Jan Dhan account have more than do...  
..                                                 ...  
195  Service charge on a food bill is not compulsor...  
196  NRIs and Indians returning from abroad will ha...  
197  Days after a Madras High Court judge indicated...  
198  Army chief General Bipin Rawat today said his ...  
199  The war within the ruling Samajwadi Party of U...  

[200 rows x 5 columns]

In [15]:
#Importing required libraries
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
import time
import csv
import json
import xml.etree.ElementTree as ET
#Creating CSV file
csv_file = open('deccan_herald.csv', 'w')
writer = csv.writer(csv_file)
writer.writerow(['Date', 'Verticle', 'Title', 'Author', 'Description'])
#Creating a empty dictionary for json file
news_dict = {}
news = 1
out_file = open("deccan_herald.json", "w")
#Creating root element for xml document
data = ET.Element('data')
#####Opening selenium Firefox session

#Turning off push notification
option = webdriver.FirefoxOptions()
option.set_preference("dom.push.enabled", False)
driver = webdriver.Firefox(options = option,executable_path=GeckoDriverManager().install())

#Opening the url
driver.get('https://www.deccanherald.com/archives')
time.sleep(5)
#Opening Datepicker
driver.find_element_by_css_selector('#arc_datepicker').click()
time.sleep(3)
#The month is March, we will select August using this loop
for i in range(1,8):
    driver.find_element_by_css_selector(".flatpickr-prev-month > svg:nth-child(1)").click()
time.sleep(5)

driver.execute_script("window.open()")
for date in range(7,38):
    #Selecting the date
    driver.find_element_by_css_selector('span.flatpickr-day:nth-child({})'.format(date)).click()
    #Opening the archive
    driver.find_element_by_css_selector('#getArchive').click()
    time.sleep(5)
    for i in range(1,10,2):
        n = 2
        try:
            content = driver.find_element_by_css_selector('#archive_result > div:nth-child({}) > div:nth-child(1) > h2:nth-child(1)'.format(i)).text
        except:
            content = driver.find_element_by_css_selector('div.grid-col-full:nth-child(3) > div:nth-child(1) > h2:nth-child(1)').text
        for j in range(1,30):
            try:
                new_dict = {}
                item = ET.SubElement(data, str(news))
                #Getting the article URL
                url = driver.find_element_by_css_selector('#archive_result > div:nth-child({}) > ul:nth-child(1) > li:nth-child({}) > a:nth-child(1)'.format(n,j)).get_attribute('href')
                #Opening a new tab and requesting the article url
                driver.switch_to.window(driver.window_handles[1])
                driver.get(url)
                time.sleep(5)
                #Scraping relevant data
                title = driver.find_element_by_css_selector('#main-wrapper > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > h1:nth-child(1)').text
                desc = driver.find_element_by_xpath('/html/body/main/div[4]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div').text
                day = driver.find_element_by_xpath('/html/body/main/div[4]/div/div/div[3]/div[2]/ul/li[1]').text
                author = driver.find_element_by_css_selector('#main-wrapper > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > a:nth-child(1)').text
                print(day)
                print(content)
                print(title)
                print(author)
                print(desc)
                print()
                #Writing the data to CSV
                writer.writerow([day, content, title, author, desc])
                #Writing the data to JSON file
                new_dict['Day'] = day
                new_dict['Verticle'] = content
                new_dict['Title'] = title
                new_dict['Author'] = author
                new_dict['Description'] = desc
                news_dict[news] = new_dict
                #Writing the data to xml document
                ET.SubElement(item, "Day").text = day
                ET.SubElement(item, "Verticle").text = content
                ET.SubElement(item, "Title").text = title
                ET.SubElement(item, "Author").text = author
                ET.SubElement(item, "Description").text = desc

                #Returning to the original window
                news = news+1
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(5)
            except:
                #If all the articles for the date are over, the loop breaks
                break
        n = n+2
    driver.find_element_by_css_selector('.flatpickr > a:nth-child(2) > img:nth-child(1)').click()
    time.sleep(3)
csv_file.close()
json.dump(news_dict, out_file, indent = 6)
out_file.close() 
tree = ET.ElementTree(data)
tree.write("filename.xml")

[WDM] - ====== WebDriver manager ======


[WDM] - Driver [/home/aditya/.wdm/drivers/geckodriver/linux64/v0.29.0/geckodriver] found in cache


AUG 01 2020, 00:40 IST
BUSINESS
Microsoft said to be in talks to buy TikTok's US operations
Bloomberg,
By Kurt Wagner
Microsoft Corp. is exploring an acquisition of TikTok’s operations in the US, according to a person familiar with the matter.
TikTok, a video-sharing app, is owned by China-based ByteDance Ltd., which has been evaluating changes to its business structure because of the US government’s national security concerns.
President Donald Trump plans to announce a decision ordering Bytedance to divest its US ownership of TikTok, according to people familiar with the matter. The US has been investigating potential national security risks due to the Chinese company’s control of the app. Trump’s decision could be announced as soon as Friday, said the people.
Microsoft is in talks to purchase TikTok’s US operations, according to the person who asked not to be identified because the discussions are private. A spokesman for Microsoft declined to comment.
ByteDance bought Musical.ly Inc

AUG 01 2020, 19:37 IST
BUSINESS NEWS
India to become No.1 in mobile manufacturing says ICEA
Ajith Athrady,
As many as 22 domestic and international firms, including iPhone maker Apple's contract manufacturers as well as Samsung, Lava and Dixon have submitted the proposal under the government's Rs 41,000-crore production-linked incentive (PLI) scheme for mobile phone manufacturing.
A total of 22 companies have filed applications under the PLI scheme from various countries including Taiwan, South Korea, German and Austria as Friday was last day for submitting proposals. Over  Rs 11 lakh crore worth mobile phones will be produced over the next five years, Union telecom and IT minister Ravi Shankar Prasad told reporters on Saturday.
However, there were no applications from Chinese companies under the scheme which was announced by the Ministry of Electronics and Information Technology to promote domestic manufacturing of smartphones and components.
"India is not averse to investment from an

AUG 01 2020, 00:40 IST
CRICKET
Microsoft said to be in talks to buy TikTok's US operations
Bloomberg,
By Kurt Wagner
Microsoft Corp. is exploring an acquisition of TikTok’s operations in the US, according to a person familiar with the matter.
TikTok, a video-sharing app, is owned by China-based ByteDance Ltd., which has been evaluating changes to its business structure because of the US government’s national security concerns.
President Donald Trump plans to announce a decision ordering Bytedance to divest its US ownership of TikTok, according to people familiar with the matter. The US has been investigating potential national security risks due to the Chinese company’s control of the app. Trump’s decision could be announced as soon as Friday, said the people.
Microsoft is in talks to purchase TikTok’s US operations, according to the person who asked not to be identified because the discussions are private. A spokesman for Microsoft declined to comment.
ByteDance bought Musical.ly Inc.

...

AUG 30 2020, 20:01 IST
ASSEMBLY ELECTION 2019
Gujarat's Covid-19 cases zoom past 95,000; deaths cross 3,000
PTI,
Gujarat on Sunday reported 1,272 new Covid-19 cases, taking the total count of infections to 95,155, state health departmentsaid.
Death toll crossed the 3,000-mark and rose to 3,008 with 17 patients succumbing to the infection, it said.
With 1,095 patients getting discharged in the day, the number of recoveries mounted to 76,757.
Follow live updates on the coronavirus here
The Covid-19 case recovery rate in the state now stands at 80.67 per cent.
A total of 69,488 samples were tested in the last 24 hours, which comes at the rate of 1,069.05 tests per day per million population, the department said in a release.

AUG 30 2020, 23:08 IST
ASSEMBLY ELECTION 2019
Himachal Pradesh reports 164 fresh Covid-19 cases, 1 more death
PTI,
Himachal Pradesh on Sunday recorded 164 fresh Covid-19 cases, taking the infection tally to 5,946, while the death toll rose to 34 with one more fatalit

AUG 30 2020, 23:08 IST
BENGALURU CRIME
Himachal Pradesh reports 164 fresh Covid-19 cases, 1 more death
PTI,
Himachal Pradesh on Sunday recorded 164 fresh Covid-19 cases, taking the infection tally to 5,946, while the death toll rose to 34 with one more fatality, an official said.
The number of active cases in the state now stands at 1,460, Additional Chief Secretary (Health) R D Dhiman said.
A 63-year-old woman in Hamirpur's Badsar died of the disease at Tanda Medical College, Kangra Chief Medical Officer Dr Gurdarshan Gupta said.
She was undergoing treatment since August 21. She also had an ovarian tumour, he added.
Kangra accounts for eight of the total Covid-19 deaths in the state, followed by Mandi and Solan (seven each), Hamirpur and Chamba (four each), Shimla (two), and Una and Sirmaur (one each).
Follow live updates on the coronavirus here
Of the 164 fresh cases, 45 were reported from Kangra, 37 from Sirmaur, 29 from Hamirpur, 16 from Solan, 12 from Chamba, ten from Shimla, nine

AUG 30 2020, 01:56 IST
BUSINESS
6-year-old boy drowns in water sump while playing
HM Chaithanya Swamy,
A six-year-old boy drowned in a water sump near his house in Kalyan Nagar, East Bengaluru, when he was playing on Friday. The victim has been identified as Ashwin, a class 1 student in a government school.
According to an investigating officer, Ashwin and a few other children were in the area on Friday afternoon. While playing, Ashwin sat on the edge of the sump. As the lid was open, he fell in. 
When there was no sign of Ashwin till around 4 pm, his father Mani went in search of him. Mani found the lid of the sump open and peered inside to see his son's floating body. With help from neighbours, they pulled out the body and alerted the police.
The Banaswadi police handed over the body to the family following the post-mortem. A case has been registered.
Mani is a vegetable vendor and has five children. Ashwin was his third son. 

AUG 30 2020, 10:44 IST
BUSINESS
PM Narendra Modi recalls

AUG 30 2020, 10:44 IST
BUSINESS NEWS
PM Narendra Modi recalls sacrifice of Imam Hussain on Muharram
PTI,
Prime Minister Narendra Modi paid homage to Imam Hussain on Sunday to mark Muharram, saying nothing was more important to him than the values of truth and justice.
Imam Hussain's emphasis on equality as well as fairness is noteworthy and gives strength to many, the Prime Minister said. 
"We recall the sacrifice of Imam Hussain (AS). For him, there was nothing more important than the values of truth and justice. His emphasis on equality as well as fairness are noteworthy and give strength to many," Modi said in a tweet. 
We recall the sacrifice of Imam Hussain (AS). For him, there was nothing more important than the values of truth and justice. His emphasis on equality as well as fairness are noteworthy and give strength to many.
— Narendra Modi (@narendramodi) August 30, 2020

AUG 30 2020, 14:48 IST
BUSINESS NEWS
Indian jails remained overcrowded and under-staffed in 2019, reveals N

AUG 31 2020, 09:31 IST
BENGALURU CRIME
Vice President M Venkaiah Naidu extends Onam greetings
PTI,
Vice President M Venkaiah Naidu on Monday greeted people on Onam, hoping that the festival brings peace, prosperity and happiness to everyone.
Onam is celebrated to honour the memory of legendary king Mahabali, the ruler of Kerala, Naidu observed.
"On this Onam, let us remind ourselves of the values of honesty, integrity, compassion, selflessness & sacrifice which the great king Mahabali espoused," the Vice President Secretariat tweeted quoting Naidu.
"May this joyous festival bring peace, prosperity and happiness to everyone," he said.

AUG 31 2020, 14:14 IST
BENGALURU CRIME
Man found dead in Greater Noida, family claims he was mentally unstable
PTI,
The body of a 34-year-old man was found along the Devla road in Uttar Pradesh's Greater Noida on Monday with his family members claiming that he was "mentally unstable", police said.
The body was noticed by the locals in the morning, who the